In [1]:
import sys
print(sys.version)

import sklearn.ensemble
import shap
import numpy as np
import numba
import time
import json
import xgboost
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
import shap.benchmark

from shap import TreeExplainer as TreeCext

from shap.explainers.pytree import *

3.10.6 (main, Nov  2 2022, 18:53:38) [GCC 11.3.0]


/home/jarek/projects/shap/venv/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
np.set_printoptions(threshold=sys.maxsize)
# X,y = shap.datasets.communitiesandcrime()
X,y = shap.datasets.boston()
print(X.columns.values.tolist())
print(len(X.columns.values.tolist()))

print(X.shape)
print(y.shape)

['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT']
13
(506, 13)
(506,)


Function load_boston is deprecated; `load_boston` is deprecated in 1.0 and will be removed in 1.2.

    The Boston housing prices dataset has an ethical problem. You can refer to
    the documentation of this function for further details.

    The scikit-learn maintainers therefore strongly discourage the use of this
    dataset unless the purpose of the code is to study and educate about
    ethical issues in data science and machine learning.

    In this special case, you can fetch the dataset from the original
    source::

        import pandas as pd
        import numpy as np

        data_url = "http://lib.stat.cmu.edu/datasets/boston"
        raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
        data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
        target = raw_df.values[1::2, 2]

    Alternative datasets include the California housing dataset (i.e.
    :func:`~sklearn.datasets.fetch_california_housing`) and the Ames housing
    datase

In [ ]:
# Zobaczmy na tym datasecie jaka glebokosc daje najlepsze wyszkolenie

X,y = shap.datasets.boston()
X = X.values
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

scores = {}
mean = 5

for i in range(1,13):
    pass
    print(i)
    scores[i] = 0
    for j in range(mean):
        model = sklearn.ensemble.RandomForestRegressor(n_estimators=1000, max_depth=i)
        model.fit(X_train, y_train)
        scores[i] = scores[i] + model.score(X_test, y_test)
    scores[i] = scores[i] / mean

scores

In [ ]:
def item_generator(json_input, lookup_key):
    if isinstance(json_input, dict):
        for k, v in json_input.items():
            if k == lookup_key:
                yield v
            else:
                yield from item_generator(v, lookup_key)
    elif isinstance(json_input, list):
        for item in json_input:
            yield from item_generator(item, lookup_key)

def tree_depth(json_text):
    json_input = json.loads(json_text)
    return max(list(item_generator(json_input, 'depth'))) + 1


In [ ]:
# Zobaczmy na tym datasecie jaka glebokosc daje najlepsze wyszkolenie

X,y = shap.datasets.boston()
X = X.values
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

scores = {}
mean_depths = {}
stddev_depths = {}

mean = 10

for i in range(1,32,32):
    print(i)
    scores[i] = 0
    mean_depths[i] = 0
    stddev_depths[i] = 0
    for j in range(mean):
        model = xgboost.XGBRegressor(n_estimators=1000, max_depth=i, subsample=0.3)
        X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)
        model.fit(X_train, y_train, eval_set=[(X_test, y_test)], early_stopping_rounds=10, verbose=False)
        scores[i] = scores[i] + model.score(X_test, y_test)
        
        booster = model.get_booster()
        tree_df = booster.trees_to_dataframe()
        print("rysuje drzewo")
        xgboost.plotting.plot_tree(booster)
        depths = [tree_depth(x) for x in booster.get_dump(dump_format = "json")]
        mean_depths[i] = mean_depths[i] + np.mean(depths)
        stddev_depths[i] = stddev_depths[i] + np.std(depths)
        
    scores[i] = scores[i] / mean
    mean_depths[i] = mean_depths[i] / mean
    stddev_depths[i] = stddev_depths[i] / mean

print(scores)
print(mean_depths)
print(stddev_depths)

In [ ]:
model = xgboost.XGBRegressor(n_estimators=1000, max_depth=5, subsample=0.3)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)
model.fit(X_train, y_train, eval_set=[(X_test, y_test)], early_stopping_rounds=10, verbose=False)

booster = model.get_booster()
tree_df = booster.trees_to_dataframe()

In [ ]:
tree_df

In [ ]:
plt.plot([x * 10 for x in scores.values()])
plt.plot(mean_depths.values())
plt.plot(stddev_depths.values())

plt.show()

# wychodza glebokosci po 13 jak da mu sie wolna reke

In [ ]:
res_shap_cext = {}
res_banz_cext = {}

time_shap_cext = {}
time_banz_cext = {}

In [ ]:
for j in range (1, 10, 2):
    if (j != 1):
        pass
    for i in range(100, 1000, 100):
        model = sklearn.ensemble.RandomForestRegressor(n_estimators=i, max_depth=13)
        model.fit(X, y)

        start = time.time()
        res_shap_cext[(i, j)] = TreeCext(model).shap_values(X, banz=False)
        time_shap_cext[(i, j)] = time.time() - start
        print(time_shap_cext[(i, j)])

        start = time.time()
        res_banz_cext[(i, j)] = TreeCext(model).shap_values(X, banz=True)
        time_banz_cext[(i, j)] = time.time() - start
        print(time_banz_cext[(i, j)])

In [3]:
def run_explain(name, explainer, data):
    start = time.time()
    values = explainer(data)
    end = time.time()
    print("name: {}, time: {}".format(name, end - start))
    return [name, values, end - start]

In [5]:
attributions = {}

for i in range(1000, 2000, 100):
    model = xgboost.XGBRegressor(n_estimators=10, max_depth=32, subsample=0.3)
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)
    model.fit(X_train, y_train, eval_set=[(X_test, y_test)], early_stopping_rounds=10, verbose=False)

    X_eval = X_test[:1]
    y_eval = y_test[:1]

    # use an independent masker
    masker = shap.maskers.Independent(X_train)
    pmasker = shap.maskers.Partition(X_train)

    explainers = [
        ("Tree", shap.explainers.Tree(model, masker)),
        ("TreeBanz", shap.explainers.Tree(model, masker, use_banz=True)),
    #     ("Tree approx.", shap.explainers.Tree(model, masker, approximate=True)),
    #     ("Exact", shap.explainers.Exact(model.predict, masker)),
    ]

    attributions[i] = [run_explain(name, exp, X_eval) for name, exp in explainers]

ile drzew cext? 10
here37
using dense_Tree_pred
ile drzew cext? 10
here37
using dense_Tree_pred
uruchamiam z BANZ=False
using treeshap and cext
here37
using dense_Tree_pred
oto values:
[[-0.02470365  0.00953999  0.30662105 -0.06799844 -0.40926368 -1.54750164
   0.13806778 -0.26119307 -0.58223099 -1.75932634 -0.43905142  0.3398083
   2.6796159 ]]
name: Tree, time: 0.000640869140625
uruchamiam z BANZ=True
using treeshap and cext
using BANZ
oto values:
[[-0.20552054  0.00785147  0.40446083 -0.0526101  -0.07763978 -1.65641686
   0.002974   -0.2731243  -0.46721783 -2.48618338 -0.05520096  0.38630166
   2.83345238]]
name: TreeBanz, time: 0.0003025531768798828
ile drzew cext? 10
here37
using dense_Tree_pred
ile drzew cext? 10
here37
using dense_Tree_pred
uruchamiam z BANZ=False
using treeshap and cext
here37
using dense_Tree_pred
oto values:
[[-0.02470365  0.00953999  0.30662105 -0.06799844 -0.40926368 -1.54750164
   0.13806778 -0.26119307 -0.58223099 -1.75932634 -0.43905142  0.3398083
   2.6

In [6]:
print(attributions)

{1000: [['Tree', .values =
array([[-0.02470365,  0.00953999,  0.30662105, -0.06799844, -0.40926368,
        -1.54750164,  0.13806778, -0.26119307, -0.58223099, -1.75932634,
        -0.43905142,  0.3398083 ,  2.6796159 ]])

.base_values =
array([20.66161737])

.data =
array([[6.7240e-02, 0.0000e+00, 3.2400e+00, 0.0000e+00, 4.6000e-01,
        6.3330e+00, 1.7200e+01, 5.2146e+00, 4.0000e+00, 4.3000e+02,
        1.6900e+01, 3.7521e+02, 7.3400e+00]]), 0.000640869140625], ['TreeBanz', .values =
array([[-0.20552054,  0.00785147,  0.40446083, -0.0526101 , -0.07763978,
        -1.65641686,  0.002974  , -0.2731243 , -0.46721783, -2.48618338,
        -0.05520096,  0.38630166,  2.83345238]])

.base_values =
array([20.66161737])

.data =
array([[6.7240e-02, 0.0000e+00, 3.2400e+00, 0.0000e+00, 4.6000e-01,
        6.3330e+00, 1.7200e+01, 5.2146e+00, 4.0000e+00, 4.3000e+02,
        1.6900e+01, 3.7521e+02, 7.3400e+00]]), 0.0003025531768798828]], 1100: [['Tree', .values =
array([[-0.02470365,  0.0095399

In [ ]:
# # dry run to get all the code warmed up for valid runtime measurements
# for name, exp in explainers:
#     exp(X_eval[:1])

# explain with all the explainers
attributions = [run_explain(name, exp, X_eval) for name, exp in explainers]

In [ ]:
X, y = make_regression(n_samples=400, n_features=16, n_informative=8,random_state=0, shuffle=False)

model = sklearn.ensemble.RandomForestRegressor(n_estimators=1000, max_depth=10)
model.fit(X, y)

In [ ]:
# bst = xgboost.train({"learning_rate": 0.01, "max_depth": 4}, xgboost.DMatrix(X, label=y), 1000)

In [ ]:
# start = time.time()
# shap_values = bst.predict(xgboost.DMatrix(X), pred_contribs=True)
# print(time.time() - start)


In [ ]:

# x = X.head()
# x = X[:1]
x = X
x

In [ ]:
start = time.time()
res_shap_cext = TreeCext(model).shap_values(x, banz=False)
time_shap_cext = time.time() - start
time_shap_cext

In [ ]:
start = time.time()
res_banz_cext = TreeCext(model).shap_values(x, banz=True)
time_banz_cext = time.time() - start
time_banz_cext

In [ ]:
ex = TreeExplainer(model)
start = time.time()
res_shap_py = ex.shap_values(x)
time_shap_py = time.time() - start

In [ ]:
start = time.time()
res_banz_py = ex.banz_values(x)
time_banz_py = time.time() - start

In [ ]:
line = np.asarray(x[:1])
ex.brute_banz(line[0], 0)

In [ ]:
ex.brute_shap(line[0], 0)

In [ ]:
results_dict_cext = {}
results_dict_cext['banz'] = {}
results_dict_cext['shap'] = {}

In [ ]:
for i in range(1, 10):
    X, y = make_regression(n_samples=200 * i, n_features=16, n_informative=8,random_state=0, shuffle=False)

    model = sklearn.ensemble.RandomForestRegressor(n_estimators=1000 + 100 * i, max_depth=10 + i)
    model.fit(X, y)
    
    start = time.time()
    res_shap_cext = TreeCext(model).shap_values(x, banz=False)
    res_shap = time.time() - start
    
    start = time.time()
    res_banz_cext = TreeCext(model).shap_values(x, banz=True)
    res_banz = time.time() - start
    
    print(res_shap)
    print(res_banz)
    results_dict_cext['banz'][i] = res_banz
    results_dict_cext['shap'][i] = res_shap
    

In [ ]:
x = [i for i in results_dict_cext['banz']]
a1 = [results_dict_cext['banz'][i] for i in results_dict_cext['banz']]
a2 = [results_dict_cext['shap'][i] for i in results_dict_cext['shap']]

y_values = {"banz": a1, "treeshap": a2}
labels = ["BANZ", "TREESHAP"]

fig, ax = plt.subplots()
ax.stackplot(x, y_values.values(), labels=labels)

ax.legend(loc='upper left')
plt.show()

In [ ]:
results_dict_py = {}
results_dict_py['banz'] = {}
results_dict_py['shap'] = {}

In [ ]:
for i in range(1, 10):
    X, y = make_regression(n_samples=100 + 25 * i, n_features=6, n_informative=2,random_state=0, shuffle=False)

    model = sklearn.ensemble.RandomForestRegressor(n_estimators=100 + 25 * i, max_depth=4)
    model.fit(X, y)
    
    ex = TreeExplainer(model)
    print('created model')
    start = time.time()
    res_shap_py = ex.shap_values(X)
    time_shap_py = time.time() - start
    
    start = time.time()
    res_banz_py = ex.banz_values(X)
    time_banz_py = time.time() - start
    
    print(res_shap_py)
    print(res_banz_py)
    results_dict_py['banz'][i] = time_banz_py
    results_dict_py['shap'][i] = time_shap_py
    

In [ ]:
x = [i for i in results_dict_py['banz']]
a1 = [results_dict_py['banz'][i] for i in results_dict_py['banz']]
a2 = [results_dict_py['shap'][i] for i in results_dict_py['shap']]

y_values = {"banz": a1, "treeshap": a2}
labels = ["BANZ", "TREESHAP"]

fig, ax = plt.subplots()
ax.stackplot(x, y_values.values(), labels=labels)

ax.legend(loc='upper left')
plt.show()

In [ ]:
len(a1)

In [ ]:
len(a2)